![Banner](../../assets/banner-sprint9-desafio.png)


## PREPARAÇÃO DOS DADOS


In [0]:
!pip install torch
!pip install spacy
!python -m spacy download en_core_web_trf
!pip install transformers==4.35.0
#!pip install gensim

     |▏                               | 3.6 MB 5.8 MB/s et

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 3.8 MB 34.8 MB/s 
     |████████████████████████████████| 464 kB 90.3 MB/s 
     |████████████████████████████████| 737 kB 65.2 MB/s 
     |████████████████████████████████| 295 kB 78.0 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-33625422-6321-4886-a28f-d2375c993a05/bin/python -m pip install --upgrade pip' command.


In [0]:
dbutils.library.restartPython()

In [0]:
from transformers import pipeline
import torch
from collections import Counter
import spacy
#from gensim import corpora, models, similarities

import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql import Window
from pyspark.sql.types import StringType, IntegerType

/local_disk0/.ephemeral_nfs/envs/pythonEnv-33625422-6321-4886-a28f-d2375c993a05/lib/python3.9/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-33625422-6321-4886-a28f-d2375c993a05/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [0]:
nlp = spacy.load("en_core_web_trf")
class_sexual = pipeline("text-classification", model="uget/sexual_content_dection")
class_sexism = pipeline("text-classification", model="annahaz/xlm-roberta-base-misogyny-sexism-indomain-mix-bal")
#ner_extract = pipeline("token-classification", model="joon09/kor-naver-ner-name", grouped_entities=True)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-33625422-6321-4886-a28f-d2375c993a05/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [0]:
filmes_tmdb_df = spark.read.parquet("dbfs:/FileStore/filmes_tmdb.parquet").alias("tmdb")
filmes_local_df = spark.read.parquet("dbfs:/FileStore/filmes_local.parquet").alias("local")
paises_df = spark.read.parquet("dbfs:/FileStore/paises.parquet")
linguas_df = spark.read.parquet("dbfs:/FileStore/linguas.parquet")

In [0]:
filmes_local_total = filmes_local_df.join(
    filmes_tmdb_df,
    filmes_tmdb_df.imdb_id == filmes_local_df.imdb_id
).select(
    "tmdb.tmdb_id",
    "local.imdb_id",
    "local.titulo_comercial",
    "tmdb.titulo_original",
    "local.ano_lancamento",
    "tmdb.lingua_original",
    "tmdb.pais_origem",
    "tmdb.popularidade",
    "local.media_avaliacao",
    "tmdb.qtd_avaliacoes",
    "tmdb.sinopse")\
.distinct()

In [0]:
filmes_df = filmes_local_total.union(filmes_tmdb_df.select(
    "tmdb_id",
    "imdb_id",
    "titulo_comercial",
    "titulo_original",
    "ano_lancamento",
    "lingua_original",
    "pais_origem",
    "popularidade",
    "media_avaliacao",
    "qtd_avaliacoes",
    "sinopse").distinct()
).filter("ano_lancamento >= 2013").drop_duplicates(["tmdb_id"])\
.orderBy("ano_lancamento")

In [0]:
filmes_df = filmes_df.withColumn(
    "texto_total", 
    F.concat(
        F.col("titulo_comercial"), 
        F.lit(" "), 
        F.col("sinopse")
    )
).alias("filmes")

## TRANSFORMAÇÕES E MODELAGEM DE DADOS

### TABELAS DIMENSIONAIS

### Dimensional: Línguas

In [0]:
linguas_dim = (linguas_df
    .withColumn("lingua", F.regexp_replace(F.col("lingua"), "Ossetian; Ossetic", "Ossetian"))
    .withColumn("lingua", F.regexp_replace(F.col("lingua"), "Haitian; Haitian Creole", "Haitian"))
    .withColumn("lingua", F.regexp_replace(F.col("lingua"), "Chichewa; Nyanja", "Chichewa"))
    .orderBy("lingua")
)

linguas_dim = linguas_dim.withColumn(
    "lingua_key",
    F.monotonically_increasing_id() + F.lit(1)
    ).select("lingua_key", "iso_cod", "lingua").alias("linguas")

### Dimensional: Países

In [0]:
regioes = {
    "North America": ["CA", "US", "GL", "UM"],
    
    "Western Europe": ["AD", "AT", "BE", "CH", "DE", "DK", "ES", "FI", "FO", "FR", "GB", "GI", "IE", 
                      "IS", "IT", "LI", "LU", "MC", "NL", "NO", "PT", "SE", "SM", "VA"],
    
    "Middle East": ["AE", "BH", "IL", "IQ", "IR", "JO", "KW", "LB", "OM", "PS", "QA", "SA", "SY",
                    "TR", "YE"],
    
    "Latin America": ["AR", "BO", "BR", "CL", "CO", "EC", "GF", "GY", "PE", "PY", "SR", "UY", "VE"],
    
    "Asia": ["AF", "AM", "AZ", "BD", "BN", "BT", "BU", "CN", "GE", "HK", "ID", "IN", "JP", "KG", 
             "KH", "KP", "KR", "KZ", "LA", "LK", "MM", "MN", "MO", "MY", "NP", "PH", "PK", 
             "SG", "TH", "TJ", "TM", "TW", "UZ", "VN"],
    
    "Polynesia": ["AS", "AU", "CC", "CK", "CX", "FJ", "FM", "GU", "HM", "KI", "MH", "MP", "NC", 
                  "NF", "NR", "NU", "NZ", "PF", "PG", "PN", "PW", "SB", "TK", "TO", "TV", "VU", "WF", "WS"],
    
    "Africa": ["AO", "BF", "BI", "BJ", "BW", "CD", "CF", "CG", "CI", "CM", "CV", "DJ", "DZ", "EG", 
               "EH", "ER", "ET", "GA", "GH", "GM", "GN", "GQ", "GW", "KE", "KM", "LR", "LS", "LY", 
               "MA", "MG", "ML", "MR", "MW", "MZ", "NA", "NE", "NG", "RW", "SC", "SD", "SL", "SN", 
               "SO", "SS", "ST", "SZ", "TD", "TG", "TN", "TZ", "UG", "ZA", "ZM", "ZR", "ZW"],
    
    "Eastern Europe": ["AL", "BA", "BG", "BY", "CS", "CZ", "EE", "GR", "HR", "HU", "LT", "LV", "MD", 
                    "ME", "MK", "PL", "RO", "RS", "RU", "SI", "SK", "SU", "UA", "XC", "XG", "XK", "YU"],
    
    "Caribbean / Central America": ["AG", "AI", "AN", "AW", "BB", "BM", "BS", "BZ", "CR", "CU",
                                    "DM", "DO", "GD", "GP", "GT", "HN", "HT", "JM", "KN", "KY", "LC", "MQ", "MS", "MX", "NI", "PA", "PR", "SV", "TC", "TT", "VC", "VG", "VI"]
}

In [0]:
@udf(returnType=StringType())
def obter_regiao(iso_cod: str) -> str:
    for regiao, paises in regioes.items():
        if iso_cod in paises:
            return regiao
    return "Unknown" 

paises_df = paises_df.withColumn(
    "regiao",
    obter_regiao(F.col("iso_cod"))
).orderBy("regiao", "pais")

paises_dim = paises_df.withColumn(
    "pais_key",
    F.monotonically_increasing_id() + F.lit(1)
    ).select("pais_key", "iso_cod", "pais", "regiao").alias("paises")

### Dimensional: Títulos

In [0]:
titulos_df = filmes_df.select(
    "titulo_comercial",
    "titulo_original",
    "tmdb_id"
).dropDuplicates(["tmdb_id"])\
.orderBy("titulo_comercial")

titulos_dim = titulos_df.withColumn(
    "titulo_key",
    F.monotonically_increasing_id() + F.lit(1)
).select("titulo_key", "titulo_comercial", "titulo_original", "tmdb_id").alias("titulos")

### Dimensional: Análise Textual

In [0]:
@udf(returnType=StringType())
def extracao_pos(doc: str, pos: str) -> list[str]:
    texto = nlp(doc)
    return [str(token.lemma_) for token in texto if token.pos_ == pos and not token.is_stop and token.is_alpha]

@udf(returnType=StringType())
def termo_mais_comum(lista: list[str]) -> str:
    if not lista:  # Tratar listas vazias
        return None
    freq_termos = Counter(lista)  # Contar ocorrências
    return max(freq_termos, key=freq_termos.get)  # Obter termo mais frequente

#### Detecção de Conteúdo Sexual

In [0]:
@udf(returnType=IntegerType())
def detector_conteudo_sexual(text):
    return int(class_sexual(str(text))[0]["label"][-1])

analise_textual_df = filmes_df.select(
    "tmdb_id", "sinopse", "texto_total"
    ).withColumn(
        "conteudo_sexual",
        detector_conteudo_sexual(F.col("texto_total"))
)

#### Detecção de Sexismo

In [0]:
@udf(returnType=IntegerType())
def detector_sexismo(text):
    return int(class_sexism(str(text))[0]["label"][-1])

analise_textual_df = analise_textual_df.withColumn(
        "sexismo",
        detector_sexismo(F.col("texto_total"))
)

#### Extração de Classificação Morfossintática

In [0]:
analise_textual_df = analise_textual_df.withColumn(
    "substantivos",
    extracao_pos(F.col("texto_total"), F.lit("NOUN"))
    ).withColumn(
        "verbos",
        extracao_pos(F.col("texto_total"), F.lit("VERB"))
    ).withColumn(
        "adjetivos",
        extracao_pos(F.col("texto_total"), F.lit("ADJ"))
    ).withColumn(
        "adverbios",
        extracao_pos(F.col("texto_total"), F.lit("ADV"))
    )

#### Extração Termos Mais Comuns

In [0]:
analise_textual_df = analise_textual_df.withColumn(
        "subst_mais_comum",
        termo_mais_comum(F.col("substantivos"))
    ).withColumn(
        "verbo_mais_comum",
        termo_mais_comum(F.col("verbos"))
    ).withColumn(
        "adj_mais_comum",
        termo_mais_comum(F.col("adjetivos"))
    ).withColumn(
        "adv_mais_comum",
        termo_mais_comum(F.col("adverbios"))
    )

In [0]:
analise_textual_dim = analise_textual_df.withColumn(
    "analise_key",
    F.monotonically_increasing_id() + F.lit(1)
).select("analise_key", "sinopse", "texto_total", "conteudo_sexual", "sexismo", "subst_mais_comum", "verbo_mais_comum", "adj_mais_comum", "adv_mais_comum", "tmdb_id").alias("analise")

### Dimensional: Corpora

In [0]:
corpora_df = analise_textual_df.groupBy() \
    .agg(F.concat_ws(" ", F.collect_list("texto_total")).alias("corpus"))

corpora_dim = corpora_df.withColumn(
    "data_registro",
    F.current_timestamp()
).withColumn(
    "corpus_key",
    F.monotonically_increasing_id() + F.lit(1)
).select("corpus_key", "corpus", "data_registro")

### Dimensional: Vocabulário

In [0]:
def split_palavras_vocab(df: object,
                         coluna_palavras: str,
                         padrao_process: str,
                         sep: str,
                         padrao_add_vocab: str,
                         coluna_extra: str) -> object:
    return df.select(
        coluna_extra,
        F.explode(
            F.split(  # Aplicação Padrão de Split de Termos
                F.regexp_replace(coluna_palavras, padrao_process, ""),
                sep
            )
        ).alias("palavra")
    ).select(
        coluna_extra,  # Aplicação Padrão de Limpeza de Termos para Inserção
        F.regexp_replace("palavra", padrao_add_vocab, "").alias("palavra")
)

padrao_process = "\\[|\\]"
padrao_add_vocab = "^\\s+|\\s+$"

In [0]:
vocab_df = analise_textual_df.select(
    F.expr("""
            stack(4,
                substantivos, 'NOUN',
                verbos, 'VERB',
                adjetivos, 'ADJ',
                adverbios, 'ADV'
            ) as (palavra, pos_class)
        """)
)

vocab_df = split_palavras_vocab(vocab_df, "palavra", padrao_process=padrao_process, sep=",", padrao_add_vocab=padrao_add_vocab, coluna_extra="pos_class")

vocab_dim = vocab_df.filter(F.col("palavra") != "").dropna().distinct().withColumn(
    "vocab_key",
    F.monotonically_increasing_id() + F.lit(1)
).select("vocab_key", "palavra", "pos_class")\
.orderBy("pos_class", "palavra").alias("vocab")

### TABELA FATO

In [0]:
filmes_fact = filmes_df.join(
    titulos_dim,
    filmes_df.tmdb_id == titulos_dim.tmdb_id
).join(
    linguas_dim,
    filmes_df.lingua_original == linguas_dim.iso_cod 
).join(
    paises_dim,
    filmes_df.pais_origem == paises_dim.iso_cod
).withColumn(
    "corpus_key",
    F.lit(1)
).withColumn(
    "filme_key",
    F.monotonically_increasing_id() + F.lit(1)
).select(
    "filme_key",
    "filmes.tmdb_id",
    "filmes.imdb_id",
    "titulos.titulo_key",
    "paises.pais_key",
    "linguas.lingua_key",
    "corpus_key",
    "filmes.ano_lancamento",
    "filmes.popularidade",
    "filmes.media_avaliacao",
    "filmes.qtd_avaliacoes").drop_duplicates(["tmdb_id"]).alias("fato")

In [0]:
filmes_fact = filmes_fact.join(
    analise_textual_dim,
    filmes_fact.tmdb_id == analise_textual_dim.tmdb_id
).select(
    "filme_key",
    "fato.tmdb_id",
    "imdb_id",
    "titulo_key",
    "pais_key",
    "lingua_key",
    "analise.analise_key",
    "corpus_key",
    "ano_lancamento",
    "popularidade",
    "media_avaliacao",
    "qtd_avaliacoes").drop_duplicates(["analise_key"]).orderBy("filme_key")


### TABELA BRIDGE VOCAB-FILMES

In [0]:
# Split de Listas de Termos Classificados Sintaticamente
palavras_df = split_palavras_vocab(
    analise_textual_df,
    "substantivos",
    padrao_process=padrao_process,
    sep=",",
    padrao_add_vocab=padrao_add_vocab,
    coluna_extra="tmdb_id"
).union(split_palavras_vocab(
    analise_textual_df,
    "verbos",
    padrao_process=padrao_process,
    sep=",",
    padrao_add_vocab=padrao_add_vocab,
    coluna_extra="tmdb_id"
)).union(split_palavras_vocab(
    analise_textual_df,
    "adjetivos",
    padrao_process=padrao_process,
    sep=",",
    padrao_add_vocab=padrao_add_vocab,
    coluna_extra="tmdb_id"
)).union(split_palavras_vocab(
    analise_textual_df,
    "adverbios",
    padrao_process=padrao_process,
    sep=",",
    padrao_add_vocab=padrao_add_vocab,
    coluna_extra="tmdb_id"
))

# Contagem de Ocorrências de Cada Palavra Normalizada
contagem_palavras = palavras_df.groupBy("tmdb_id", "palavra").agg(
    F.count("*").alias("frequencia")
)

# JOIN Entre Fato Filmes e Dimensional Vocabulário
filmes_vocab_bridge = contagem_palavras.join(
    vocab_dim,
    contagem_palavras.palavra == vocab_df.palavra,
    "inner"
).join(
    filmes_fact,
    filmes_fact.tmdb_id == contagem_palavras.tmdb_id,
    "inner"
).select(
    "fato.filme_key",
    "vocab.vocab_key",
    F.col("frequencia")
)


In [0]:
#display(filmes_vocab_bridge.filter(F.col("filme_key") == 29))

### DROP DE COLUNAS AUXILIARES

In [0]:
filmes_vocab_bridge = filmes_vocab_bridge.drop("tmdb_id")
titulos_dim = titulos_dim.drop("tmdb_id")
analise_textual_dim = analise_textual_dim.drop("tmdb_id")

## ANÁLISES EXPLORATÓRIAS DIVERSAS

### Quantidade de Linhas

In [0]:
filmes_fact.select("filme_key").count()

Out[26]: 484

### Frequência de Países

In [0]:
display(filmes_fact.join(
    paises_dim,
    filmes_fact.pais_key == paises_dim.pais_key,
    "left"
).select("fato.pais_key", "iso_cod", "pais", "regiao")\
    .groupby("pais", "regiao") \
    .count()
    .orderBy(F.col("count").desc())
)

pais,regiao,count
South Korea,Asia,425
Japan,Asia,19
Philippines,Asia,10
China,Asia,9
India,Asia,7
Russia,Eastern Europe,3
Thailand,Asia,2
Turkey,Middle East,2
Ethiopia,Africa,1
Brazil,Latin America,1


### Quantidade Filmes Conteúdo Sexual por País

In [0]:
display(filmes_fact.join(
    paises_dim,
    filmes_fact.pais_key == paises_dim.pais_key,
    "left"
).join(
    analise_textual_dim,
    filmes_fact.analise_key == analise_textual_dim.analise_key
).select("pais", "conteudo_sexual")\
    .filter(F.col("conteudo_sexual") == 1)
    .groupby("pais", "conteudo_sexual") \
    .count()
    .orderBy(F.col("count").desc())
)

pais,conteudo_sexual,count
South Korea,1,305
Japan,1,11
Philippines,1,7
China,1,5
India,1,5
Russia,1,3
Argentina,1,1
Bangladesh,1,1
Turkey,1,1
Mexico,1,1


### Quantidade de Filmes Lançados por Ano / País (Filtro Conteúdo Sexual)

In [0]:
display(filmes_fact.join(
    paises_dim,
    filmes_fact.pais_key == paises_dim.pais_key,
    "left"
).join(
    analise_textual_dim,
    analise_textual_dim.analise_key == filmes_fact.analise_key
).select("fato.ano_lancamento", "analise.conteudo_sexual", "paises.pais", "paises.regiao")\
    .groupby("fato.ano_lancamento", "paises.pais", "analise.conteudo_sexual", "paises.regiao") \
    .count() \
    .orderBy(F.col("count").desc()) \
)

ano_lancamento,pais,conteudo_sexual,regiao,count
2020,South Korea,1,Asia,59
2021,South Korea,1,Asia,54
2018,South Korea,1,Asia,50
2019,South Korea,1,Asia,47
2020,South Korea,0,Asia,26
2023,South Korea,1,Asia,24
2019,South Korea,0,Asia,17
2016,South Korea,1,Asia,17
2017,South Korea,0,Asia,16
2017,South Korea,1,Asia,16


### Línguas Distintas

In [0]:
display(filmes_fact.select("lingua_key").distinct().join(
    linguas_dim,
    filmes_fact.lingua_key == linguas_dim.lingua_key,
    "left"
).select("lingua"))

lingua
Bengali
Portuguese
Arabic
Tagalog
Amharic
Korean
Mandarin
Hindi
Telugu
Malayalam


In [0]:
display(filmes_fact.join(
        linguas_dim,
        filmes_fact.lingua_key == linguas_dim.lingua_key,
        "left"
    ).join(
        paises_dim,
        filmes_fact.pais_key == paises_dim.pais_key,
        "left"
    )
    .select("pais", "lingua")
    .groupBy("pais", "lingua")
    .agg(
        F.count("*").alias("quantidade")
    )
    .orderBy(F.col("quantidade").desc())
)

pais,lingua,quantidade
South Korea,Korean,425
Japan,Japanese,19
Philippines,Tagalog,10
China,Mandarin,9
India,Hindi,3
India,Telugu,3
Russia,Russian,3
Thailand,Thai,2
Turkey,Turkish,2
Egypt,Arabic,1


### Frequência de Lançamentos por Ano

In [0]:
display(
    filmes_fact.join(
        paises_dim,
        filmes_fact.pais_key == paises_dim.pais_key,
        "left"
    )
    .groupBy("ano_lancamento", "pais")
    .agg(
        F.count("*").alias("quantidade")
    )
    .orderBy(F.col("ano_lancamento").desc(), F.col("quantidade").desc())
)

ano_lancamento,pais,quantidade
2025,Thailand,1
2025,Ethiopia,1
2025,Turkey,1
2024,South Korea,22
2024,Philippines,3
2024,Japan,1
2024,Bangladesh,1
2024,India,1
2024,Thailand,1
2023,South Korea,35
